In [19]:
from sklearn.preprocessing import LabelEncoder, scale
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd

## open train and test data

In [5]:
train_csv = pd.read_csv('titanic/train.csv')
test_csv = pd.read_csv('titanic/test.csv')

train_csv.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## tranform data for machine learning

In [36]:
embarked_lb = LabelEncoder()
embarked_lb.fit(train_csv['Embarked'].fillna('').values)

sex_lb = LabelEncoder()
sex_lb.fit(train_csv['Sex'].values)


def transform(df):
    """transform dataframe"""    
    p_class = df['Pclass'].values
    
    sex = sex_lb.transform(df['Sex'].values)
    
    sib_sp = scale(df['SibSp'].values)
    
    parch = scale(df['Parch'].values)
    
    fare = scale(df['Fare'].fillna(0).values)
    
    have_cabin = [1 if i else 0 for i in df['Cabin'].values]
    
    age = scale(df['Age'].fillna(0).values)
    
    embarked = embarked_lb.transform(df['Embarked'].fillna('').values)
    
    return pd.DataFrame({
        'p_class': p_class,
        'sex': sex,
        'age': age,
        'sib_sp': sib_sp,
        'parch': parch,
        'fare': fare,
        'have_cabin': have_cabin,
        'embarked': embarked
    })


train = transform(train_csv)
label = train_csv['Survived'].values
x_train, x_valid, y_train, y_valid = train_test_split(train, label, test_size=0.20, random_state=42, stratify=label)

/home/faisal/kaggle/.venv/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


## Classifier

In [41]:
clf = SVC(kernel='linear', class_weight=None, gamma='auto')
clf.fit(x_train, y_train)
clf.score(x_valid, y_valid)

0.77653631284916202

## Predict

In [38]:
test = transform(test_csv)

result_df = pd.DataFrame({
    "PassengerId": test_csv["PassengerId"].values,
    "Survived": clf.predict(test) 
})
result_df.to_csv("titanic/predictions.csv", index=False)
result_df.head()

/home/faisal/kaggle/.venv/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [27]:
clf.best_params_

{'C': 1000.0}